In [ ]:
import sqlite3
import re

# Create or connect to an SQLite database
conn = sqlite3.connect("company.db")  
cursor = conn.cursor()

# Create the Employees and Departments tables
def create_tables():
    cursor.execute("DROP TABLE IF EXISTS Employees")
    cursor.execute("DROP TABLE IF EXISTS Departments")

    cursor.execute('''CREATE TABLE Employees (
                        ID INTEGER PRIMARY KEY AUTOINCREMENT,
                        Name TEXT NOT NULL,
                        Department TEXT NOT NULL,
                        Salary REAL NOT NULL,
                        Hire_Date TEXT NOT NULL)''')
    
    cursor.execute('''CREATE TABLE Departments (
                        ID INTEGER PRIMARY KEY AUTOINCREMENT,
                        Name TEXT NOT NULL UNIQUE,
                        Manager TEXT NOT NULL)''')
    conn.commit()

# Insert sample data into both tables
def insert_sample_data():
    employees = [
        ("Alice", "Sales", 50000, "2021-01-15"),
        ("Bob", "Engineering", 70000, "2020-06-10"),
        ("Charlie", "Marketing", 60000, "2022-03-20"),
    ]
    
    departments = [
        ("Sales", "Alice"),
        ("Engineering", "Bob"),
        ("Marketing", "Charlie"),
    ]

    cursor.executemany("INSERT INTO Employees (Name, Department, Salary, Hire_Date) VALUES (?, ?, ?, ?)", employees)
    cursor.executemany("INSERT INTO Departments (Name, Manager) VALUES (?, ?)", departments)
    conn.commit()

# Function to execute queries
def execute_query(query, params=None, fetch=False):
    try:
        if params:
            cursor.execute(query, params)
        else:
            cursor.execute(query)
        
        if fetch:
            return cursor.fetchall()
        else:
            conn.commit()
    except sqlite3.Error as e:
        print(f"Database error: {e}")
        return None

# Function to handle user queries
def handle_user_query(query):
    query = query.lower().strip()

    # Query total salary expense in a department
    match = re.match(r'.*salary.*expense.*(?:in|of|for)?\s+(\w+)\s+department', query)
    if match:
        department = match.group(1).capitalize().strip()
       
        sql_query = "SELECT SUM(Salary) FROM Employees WHERE Department = ?"
        result = execute_query(sql_query, (department,), fetch=True)

        if result and result[0][0] is not None:
            return f"The total salary expense for the {department} department is ${result[0][0]:,.2f}."
        else:
            return f"No salary data found for the {department} department."

    # Query listing employees in a department
    match = re.match(r'.*employees.*(?:in|of|for)?\s+(\w+)\s+department', query)
    if match:
        department = match.group(1).capitalize().strip()

        sql_query = "SELECT Name FROM Employees WHERE Department = ?"
        result = execute_query(sql_query, (department,), fetch=True)

        if result:
            employees = ', '.join(row[0] for row in result)
            return f"Employees in the {department} department: {employees}."
        else:
            return f"No employees found in the {department} department."

    # Query for department managers
    match = re.match(r'.*manager.*(?:of|for)?\s+(\w+)\s+department', query)
    if match:
        department = match.group(1).capitalize().strip()

        sql_query = "SELECT Manager FROM Departments WHERE Name = ?"
        result = execute_query(sql_query, (department,), fetch=True)

        if result:
            return f"The manager of the {department} department is {result[0][0]}."
        else:
            return f"No manager found for the {department} department."

    return "Sorry, I didn't understand that query."

# Function to run the chat assistant
def chat_assistant():
    print("Welcome to the Employee Database Assistant!")
    print("You can ask about salary expenses, list employees, or find department managers.")

    while True:
        user_query = input("\nEnter your query (or 'exit' to quit): ")
        if user_query.lower() == 'exit':
            print("Goodbye!")
            break

        response = handle_user_query(user_query)
        print(response)

# Initialize database
create_tables()
insert_sample_data()

# Run the assistant
chat_assistant()


Welcome to the Employee Database Assistant!
You can ask about salary expenses, list employees, or find department managers.



Enter your query (or 'exit' to quit):  display all employees of sales department


Employees in the Sales department: Alice.
